# Google Colab: Access Webcam for Images and Video
This notebook will go through how to access and run code on images and video taken using your webcam.  

For this purpose of this tutorial we will be using OpenCV's Haar Cascade to do face detection on our Webcam image and video.

In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

## Helper Functions
Below are a few helper function to make converting between different image data types and formats.

In [2]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

## Haar Cascade Classifier
For this tutorial we will run a simple object detection algorithm called Haar Cascade on our images and video fetched from our webcam. OpenCV has a pre-trained Haar Cascade face detection model.

In [3]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

## Webcam Images
Running code on images taken from webcam is fairly straight-forward. We will utilize code within Google Colab's **Code Snippets** that has a variety of useful code functions to perform various tasks.

We will be using the code snippet for **Camera Capture** to utilize your computer's webcam.

In [4]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # # get face bounding box coordinates using Haar Cascade
  # faces = face_cascade.detectMultiScale(gray)
  # # draw face bounding box on image
  # for (x,y,w,h) in faces:
  #     img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # save image
  cv2.imwrite(filename, img)

  return filename, img

In [5]:
count = 4
images = []

def capture_images(cnt):
    try:
        filename, img = take_photo(f'photo_{cnt}.jpg')
        print('Saved to {}'.format(filename))

        # Show the image which was just taken.
        display(Image(filename))
        return img
    except Exception as err:
        # Errors will be thrown if the user does not have a webcam or if they do not
        # grant the page permission to access it.
        print(str(err))
        return None


In [6]:
def save_combine_images_vertically(images, padding=10, gap=10):
    if len(images) != 4:
        print("이미지가 4장이 아닙니다.")
        return None
    # 각 이미지의 크기를 구합니다
    height,  width, _ = images[0].shape

    # 패딩이 적용된 이미지 크기
    padded_height = height + 2 * padding
    padded_width = width + 2 * padding

    # 세로로 결합할 때 전체 높이를 계산합니다 (간격 포함)
    combined_height = padded_height * 4 + gap * 3

    # 빈 이미지(검은색 배경)를 만듭니다
    combined_image = np.zeros((combined_height, padded_width, 3), dtype=np.uint8)

    # 이미지를 배치할 때 패딩과 간격을 적용합니다
    for i in range(4):
        y_offset = i * (padded_height + gap)
        combined_image[y_offset + padding:y_offset + padding + height, padding:padding + width] = images[i]

    cv2.imwrite('/content/combine_images_vertically.jpg', combined_image)
    print('성공적으로 이미지를 저장했습니다!')

# 네컷 촬영 하기

In [7]:
def main():
    for i in range(1, 5):
        img = capture_images(i)
        images.append(img)
    # print(f'images : {images}')
    if len(images) == 4:
        save_combine_images_vertically(images)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
def load_and_modify_image(file_path):
    # 이미지 파일을 불러옵니다
    combined_image = cv2.imread(file_path)

    if combined_image is None:
        print(f"{file_path} 파일을 읽을 수 없습니다.")
        return

    # 검정색 배경을 흰색으로 변경합니다
    combined_image[np.where((combined_image == [0,0,0]).all(axis=2))] = [255, 255, 255]

    return combined_image

In [ ]:
def main():
    # 저장된 결합된 이미지 파일 경로
    file_path = '/content/combine_images_vertically.jpg'

    # 이미지를 불러와서 수정합니다
    modified_image = load_and_modify_image(file_path)

    if modified_image is not None:
        # 수정된 이미지를 저장합니다 (예를 들어, 'modified_image.jpg'로 저장)
        cv2.imwrite('/content/modified_background.jpg', modified_image)
        print("수정된 이미지를 'modified_image.jpg'로 저장했습니다.")

In [ ]:
if __name__ == "__main__":
    main()

# 흑백 사진으로 변형하기

In [ ]:
def load_and_modify_image(file_path):
    # 이미지 파일을 불러옵니다
    combined_image = cv2.imread(file_path)

    if combined_image is None:
        print(f"{file_path} 파일을 읽을 수 없습니다.")
        return

    # 검정색 배경을 흰색으로 변경합니다
    combined_image[np.where((combined_image == [0,0,0]).all(axis=2))] = [255, 255, 255]

    # 이미지를 흑백으로 변환합니다
    gray_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2GRAY)

    return gray_image

def main():
    # 저장된 결합된 이미지 파일 경로
    file_path = '/content/combine_images_vertically.jpg'

    # 이미지를 불러와서 수정합니다
    gray_image = load_and_modify_image(file_path)

    if gray_image is not None:
        # 수정된 이미지를 저장합니다 (예를 들어, 'modified_image.jpg'로 저장)
        cv2.imwrite('/content/modified_gray_image.jpg', gray_image)
        print("수정된 이미지를 'modified_gray_image.jpg'로 저장했습니다.")

In [ ]:
if __name__ == "__main__":
    main()

# 이미지에 텍스트 추가하기

In [ ]:
def load_and_modify_image(file_path):
    # 이미지 파일을 불러옵니다
    combined_image = cv2.imread(file_path)

    if combined_image is None:
        print(f"{file_path} 파일을 읽을 수 없습니다.")
        return

    # 검정색 배경을 흰색으로 변경합니다
    combined_image[np.where((combined_image == [0,0,0]).all(axis=2))] = [255, 255, 255]

    # 이미지에 텍스트를 추가합니다
    text = "Hello, OpenCV!"
    org = (50, 50)  # 텍스트의 좌측 상단 시작 위치
    fontFace = cv2.FONT_HERSHEY_SIMPLEX  # 폰트 선택
    fontScale = 1  # 폰트 크기
    color = (255, 0, 0)  # 텍스트 색상 (BGR 형식)
    thickness = 2  # 선 두께

    combined_image = cv2.putText(combined_image, text, org, fontFace, fontScale, color, thickness, cv2.LINE_AA)

    return combined_image

def main():
    # 저장된 결합된 이미지 파일 경로
    file_path = '/content/combine_images_vertically.jpg'

    # 이미지를 불러와서 수정합니다
    combined_image = load_and_modify_image(file_path)

    if combined_image is not None:
        # 수정된 이미지를 저장합니다 (예를 들어, 'modified_image.jpg'로 저장)
        cv2.imwrite('/content/modified_text_image.jpg', combined_image)
        print("텍스트가 추가된 이미지를 'modified_text_image.jpg'로 저장했습니다.")

In [ ]:
if __name__ == "__main__":
    main()